In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

import datetime


In [2]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"} 

In [3]:
URL = 'https://www.lastampa.it'

In [4]:
page = requests.get(URL, headers=headers)

In [5]:
# Parse the HTML content
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
# Seleziona l'elemento
articles = soup.select("article[class^='entry default']")

In [14]:
title = articles[1].select_one('h2').text.strip()
print(title)

Ecco la Quarta guerra mondiale: l’Ucraina è la prima linea europea


In [20]:
link = articles[1].select_one('a')['href']
print(link)

https://www.lastampa.it/esteri/2023/01/07/news/lucraina_e_la_prima_linea_europea_della_quarta_guerra_mondiale-12451315/


In [22]:
article = link
article_page = requests.get(article, headers=headers)
article_soup = BeautifulSoup(article_page.content, 'html.parser')
text = article_soup.select_one("div[class='story__content']").text.strip()
print(text)


Abbonati per leggere anche


In [35]:
for i in range(len(articles)):
    try:
       imagine = articles[i].select_one("source")['data-srcset']
    except:
        pass


In [27]:
img = articles[0].select_one("source")['data-srcset']
print(img)

KeyError: 'data-srcset'

In [36]:
# Crea una lista vuota che conterrà i dati estratti
data = []

# Itera su ogni elemento
for i in range(len(articles)):
    # Estrai il titolo
    title = articles[i].select_one('h2').text.strip()

    # Estrai il link
    link = articles[i].select_one('a')['href']

    #estrai immagine
    try:
        img = articles[i].select_one("source")['data-srcset']
    except:
        try:
            img = articles[i].select_one("img")['src']
        except:
            pass

    #estraggo il testo
    try:
        article = link
        article_page = requests.get(article, headers=headers)
        article_soup = BeautifulSoup(article_page.content, 'html.parser')
        text = article_soup.select_one("div[class='story__content']").text.strip()
    except:
        pass

    # Aggiungi i dati estratti alla lista
    data.append({
        'title': title,
        'image': img,
        'link': link,
        'text': text
    })

# Crea un DataFrame Pandas a partire dalla lista di dati
df = pd.DataFrame(data)

In [37]:
df['date'] = datetime.datetime.today().strftime("%Y-%m-%d")

In [38]:
df.set_index('date', inplace=True)

In [39]:
df.tail()

,title,image,link,text
date,,,,
2023-01-07,Se il pubblico non sa più garantire l’assistenza,https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/06/news...,Abbonati per leggere anche
2023-01-07,Dopo tre anni Ivrea ritrova il suo Carnevale,https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/06/news...,Abbonati per leggere anche
2023-01-07,"Via le ultime impalcature, piazza San Carlo to...",https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/06/news...,Da poche ore piazza San Carlo è tornata libera...
2023-01-07,Basta “Pasta e basta”: l’ultima notte di Nicol...,https://www.gedistatic.it/content/gnn/img/last...,https://www.lastampa.it/torino/2023/01/05/news...,Abbonati per leggere anche
2023-01-07,Ecco la selezione delle migliori offerte della...,https://www.gedistatic.it/content/gnn/img/last...,https://www.consigli.it/migliori-offerte-del-g...,Abbonati per leggere anche


In [42]:
df.to_csv('data/daily_news.csv', mode='a', header=False)